In [1]:
#get artists as array elements:
import os , json , spotipy , pandas as pd, numpy as np , csv , time , scipy, networkx as nx , karateclub
from node2vec import Node2Vec
from functools import reduce
from spotipy.oauth2 import SpotifyClientCredentials
from pprint import pprint
from node2vec.edges import HadamardEmbedder
SPOTIPY_CLIENT_ID = "8fd508b6bab54ba1bb64085c7e9872f2"
SPOTIPY_CLIENT_SECRET = "7fb7175f176747f298030d5fb53f1140"

c:\Users\ujjwa\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# code which convert each artist into their encoding number, this will help in node2vec

# now for edgelist creation

In [127]:
#get artists as array elements:
import os , json , spotipy , pandas as pd, numpy as np , csv , time , scipy, networkx as nx , karateclub
from node2vec import Node2Vec
from functools import reduce
from spotipy.oauth2 import SpotifyClientCredentials
from pprint import pprint
from node2vec.edges import HadamardEmbedder
SPOTIPY_CLIENT_ID = "64201cf2e1cd402c8830f01d80b87a86"
SPOTIPY_CLIENT_SECRET = "12f2ff4c90134fa1a9743dbb5c79a72f"


"""Requires two files"""
class ArtistEncoder():
    def __init__(self ,spotify_creds : list[str] ,artist_name_id_file_path, artist_edgelist_file_path):
        self._spotify_client_id = spotify_creds[0]
        self._spotify_client_secret = spotify_creds[1]
        self.artist_name_id_file_path= artist_name_id_file_path
        self.artist_edgelist_file_path = artist_edgelist_file_path

        
        f1 = open(self.artist_name_id_file_path,'r',encoding='utf-8')
        f2 = open(self.artist_name_id_file_path,'r',encoding='utf-8')
        f3 = open(self.artist_name_id_file_path,'r',encoding='utf-8')
        self._artistnames = [artist_details.split('|')[0] for artist_details in f1.readlines()]
        self._artistids = [artist_details.split('|')[1].rstrip("\n") for artist_details in f3.readlines()]
        self.name2edgenum = {aname.split("|")[0]:idx for idx , aname in enumerate(f2.readlines())}
        self.edgenum2name = {v:k for k,v in self.name2edgenum.items()}
        f1.close()
        f2.close()
        f3.close()

        self._artistedges = json.loads(open(artist_edgelist_file_path,'r').read())
        if 'numerical_edgelist' not in self._artistedges:
            self._artistedges['numerical_edgelist'] = [[self.name2edgenum[a] , self.name2edgenum[b]] for a,b in self._artistedges['named_edgelist']]
            

        if artist_edgelist_file_path is not None and artist_name_id_file_path is not None:
            self.vectorize()
        
    """if artist name file doesnt exist then it creates it from song data file"""
    def build_artist_name_id_file(self,songdatacsv_file_path : str , reset = False):
        if len(self._artistnames) > 0 and reset is False:
            print("Artist Id file already built, to replace, use reset = True")
            return
        self.artist_name_id_file_path = 'artist-names.txt'
        with open('artist-names.txt','w',encoding='utf-8') as f: 
            v3 = sorted(list(set(map(lambda x : x[0]+ "|"+x[1]+"\n",
                                    reduce(lambda x,y: x+y ,
                                            [list(zip(eval(art),eval(art_id))) for art, art_id in 
                                            pd.read_csv(songdatacsv_file_path).loc[:,["artists","artist_ids"]].values])))))
            f.writelines(v3)
        print(f"Artist List Built as {self.artist_name_id_file_path}")
        f1 = open(self.artist_name_id_file_path,'r',encoding='utf-8')
        f2 = open(self.artist_name_id_file_path,'r',encoding='utf-8')
        self._artistnames = [artist_details.split('|')[0] for artist_details in f1.readlines()]
        self._artistids = [artist_details.split('|')[1] for artist_details in f2.readlines()]
        f1.close()
        f2.close()
        self.name2edgenum = {aname.split("|")[0]:idx for idx , aname in enumerate(open(self.artist_name_id_file_path,'r',encoding='utf-8').readlines())}
        self.edgenum2name = {v:k for k,v in self.name2edgenum.items()}

    """if edgelist doesnt exist it builds from spotify and artist name file"""
    def build_edgelist(self,edgelist_file_name,reset=False):
        if len(self._artistedges.keys()) > 0 and reset is False:
            print("Edgelist file already built, to replace, use reset = True")
            return
        csdata = csv.reader(open(self.artist_name_id_file_path,'r',encoding='utf-8'),delimiter='|')
        self.sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(self._spotify_client_id,self._spotify_client_secret))
        edgelist = []
        temp_edges = []
        for idx , (artistname, artistid) in enumerate(csdata):
            print(f"Sn. {idx+1} {artistname:35},{artistid:24}", end=" ")
            while True:
                data = self.sp.artist_related_artists(artist_id=artistid)
                if 'error' in data.keys():
                    print(data['error']['status'])
                    time.sleep(3)
                    continue
                filtered_data = [artist['name'] for artist in data['artists'] if artist['name'] in self.name2edgenum]
                temp_edges += list(map(lambda x: sorted([artistname,x]) , filtered_data))
                print(f"{len(filtered_data)} entries ", filtered_data )
                if (idx%10 == 0):
                    edgelist = edgelist + temp_edges
                    temp_edges = []
                break  
        edgelist += temp_edges
        self._artistedges['named_edgelist'] = edgelist
        self._artistedges['numerical_edgelist'] = [[self.name2edgenum[a] , self.name2edgenum[b]] for a,b in edgelist]
        with open(edgelist_file_name,'w') as f: f.write(json.dumps(self._artistedges,indent=2))

    """basic access functions"""
    def get_named_edgelist(self):
        return self._artistedges['named_edgelist']
    def get_numerical_edgelist(self):
        return self._artistedges['numerical_edgelist']
    def save_edgelist(self,edgelist_file_name):
        with open(edgelist_file_name,'w') as f :f.write(json.dumps(self._artistedges,indent=2))
        print(f"Saved successfully in {edgelist_file_name}")

    """Vectorizer"""
    def vectorize(self):
        self.graph = nx.Graph()
        self.graph.add_nodes_from(range(len(self.edgenum2name)))
        self.graph.add_edges_from(self._artistedges['numerical_edgelist'])
        dp = nx.all_pairs_shortest_path(enc.graph,cutoff=10)
        self.artist_distance_information = {cnt[0]:cnt[1] for cnt in dp}
        
    """Similarity Score Function (1 is best) """
    def getSimilarityLoss(self, spotify_id1 : str = None , spotify_id2 : str = None):
        translate = lambda x : np.exp(-x/6)
        artist_num1 = self._artistids.index(spotify_id1)
        artist_num2 = self._artistids.index(spotify_id2)
        if artist_num2 in self.artist_distance_information[artist_num1]:
            return 1-translate(len(self.artist_distance_information[artist_num1][artist_num2]))
        else: return 1-np.random.uniform(0.08,0.11)        
    


"""Example"""
enc = ArtistEncoder([SPOTIPY_CLIENT_ID,SPOTIPY_CLIENT_SECRET],'artist-names.txt','edges.json')
print(enc.getSimilarityLoss('0C0XlULifJtAgn6ZNCW2eu' , '2YZyLoL8N0Wb9xBt1NhZWg')) #drake x kendrick
print(enc.getSimilarityLoss('3aQeKQSyrW4qWr35idm0cy' , '6DPYiyq5kWVQS4RGwxzPC7')) #drake x dr dree
print(enc.getSimilarityLoss('0C0XlULifJtAgn6ZNCW2eu' , '3aQeKQSyrW4qWr35idm0cy')) #drake x avamax
print(enc.getSimilarityLoss('06HL4z0CvFAxyc27GXpf02' , '0C0XlULifJtAgn6ZNCW2eu')) #taylor x avamax

In [192]:
class GenreRecommender():
    def __init__(self, song_data_filepath : str):
        self.song_data_filepath = song_data_filepath
        with open(self.song_data_filepath,'r',encoding='utf-8') as f: self.song_data = json.loads(f.read())

        self.song_genre_dict = { d['SongID'] : list(set(d['SongData']['genre']+d['SongData']['artists'][0]['genre'])) for d in self.song_data}
        self.songs = sorted(list(set([x['SongID'] for x in self.song_data])))
        self.genres = sorted(list(set(reduce(lambda x,y: x+y , [x['SongData']['genre'] + x['SongData']['artists'][0]['genre'] for x in self.song_data]))))
        self.num_songs = len(self.songs)
        self.num_genres = len(self.genres)

        self.graph = nx.Graph()
        self.graph.add_nodes_from(self.songs)
        self.graph.add_nodes_from(self.genres)
        for a,b in self.song_genre_dict.items():
            for g in b:
                self.graph.add_edge(a,g)
    def hasCommonGenre(self,songid1 , songid2):
        a1 = set(self.graph.neighbors(songid1))
        a2 = set(self.graph.neighbors(songid2))
        return 1.5 * np.exp(-3/2*len(a1.intersection(a2)))
        
genre_rec = GenreRecommender("filtered_songdata.json")


genre_rec.hasCommonGenre('0VjIjW4GlUZAMYd2vXMi3b', '1bDbXMyjaUIooNwFE9wn0N')

In [1]:
#get artists as array elements:
import os , json , spotipy , pandas as pd, numpy as np , time , networkx as nx 
from functools import reduce
from spotipy.oauth2 import SpotifyClientCredentials
from pprint import pprint

SPOTIPY_CLIENT_ID = "19abf223b162460bb51c3e9976684416"
SPOTIPY_CLIENT_SECRET = "619ac59e4b174c8db1477fff87c0f3a5"

In [14]:
class ArtistRecommender():
    def __init__(self,song_data_filepath):
        self.song_data_filepath = song_data_filepath
        with open(self.song_data_filepath,'r',encoding='utf-8') as f: self.song_data = json.loads(f.read())
        self.song_artist_dict = { d['SongID'] : [artistdet['id'] for artistdet in d['SongData']['artists']] for d in self.song_data}
        self.artist_ids = set(reduce(lambda x,y:x+y , [[artistdet['id'] for artistdet in d['SongData']['artists']] for d in self.song_data]))
        self.artist_relations = None

    def getArtistRelationFromSpotify(self,spotify_creds,savefilename):
        sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(spotify_creds[0],spotify_creds[1]))
        with open(savefilename,'r+',encoding='utf-8') as filesaver : self.edgelist = json.loads(filesaver.read())['edges']
        print("EdgeList loaded : ", len(self.edgelist))
        temp_edges = []
        for idx , artistid in enumerate(self.artist_ids):
            if idx<=15000: continue
            print(f"Sn. {idx+1} ,{artistid:24}", end=" ")
            while True:
                data = sp.artist_related_artists(artist_id=artistid)
                if 'error' in data.keys():
                    print(data,flush=True)
                    print(data['error']['status'],flush=True)
                    time.sleep(3)
                    continue
                filtered_data = [d['id'] for d in data['artists'] if d['id'] in self.artist_ids]
                temp_edges += list(map(lambda x: sorted([artistid,x]) , filtered_data))
                print(f"{len(filtered_data)} entries ", filtered_data )
                if (idx%10 == 0):
                    self.edgelist = self.edgelist + temp_edges
                    temp_edges = []
                    with open(savefilename,'w',encoding='utf-8') as f: f.write(json.dumps({'edges':self.edgelist}))
                break  
        self.edgelist += temp_edges
        with open(savefilename,'w',encoding='utf-8') as f: f.write(json.dumps({'edges':self.edgelist}))
        

    def getArtistRelationFromFile(self,filename):
        with open(filename,'r',encoding='utf-8') as f: self.artist_relations = json.loads(f.read())
        self.graph = nx.Graph()
        self.graph.add_nodes_from(self.artist_ids)
        self.graph.add_edges_from(self.artist_relations['edges'])

    def artistLoss(self,artistid1,artistid2):
        translate = lambda x : np.exp(-x/6)
        if nx.has_path(artist_rec.graph,e1,e2):
            return translate(nx.shortest_path_length(artist_rec.graph,e1,e2))
        else: return np.random.uniform(1e-2) 

artist_rec = ArtistRecommender('filtered_songdata.json')
artist_rec.getArtistRelationFromFile('artist_edges.json')
e1 = '6eUKZXaKkcviH0Ku9w2n3V'
e2 = '6eUKZXaKkcviH0Ku9w2n3V'
artist_rec.artistLoss(e1,e2)

In [16]:
# artist_rec.getArtistRelationFromSpotify([SPOTIPY_CLIENT_ID,SPOTIPY_CLIENT_SECRET],'artist_edges.json')

# artist_rec.artist_relations['edges']



Computing Path Information. Please Wait


1.0